# 02 Service Lifetimes

In the examples so far, all of the services created have been "transient"; this means that when they are requested from the `ServiceContainer` a new instance is created each time. So, (under normal circumstances...) any modifications to the dependency service are not apparent to any other service which use it.  

For example, if we have a services `IFoo`, `IBar` and `IBaz`, where `IBar` and `IBaz` are dependent on `IFoo`.
If the `IFoo` service is transient, then `IBar` makes a modification to `IFoo`. `IBaz` will not be aware of the modifications because `IBar` and `IBaz` each have their own instances of `IFoo`. 

"Transient" is used to describe the "LifeTime" of the service. When services are registered with a `ServiceContainer`, the default lifetime is "transient". You can specify other lifetimes, which will be discussed later.

Before checking out other lifetimes, let's see a working example of the issue described...

In [1]:
from abc import ABC, abstractmethod

from pyjudo import ServiceContainer

In [2]:
# Interfaces
class IFoo(ABC):
    text: str

    @abstractmethod
    def foo(self): ...

class IBar(ABC):
    @abstractmethod
    def bar(self): ...

class IBaz(ABC):
    @abstractmethod
    def baz(self): ...


# Implementations
class Foo(IFoo):
    def __init__(self, text: str = "default text"):
        self.text = text

    def foo(self):
        print(self.text)

class Bar(IBar):
    def __init__(self, foo: IFoo):
        self.foo = foo

    def bar(self):
        print(f"I'm a Bar and my Foo says: {self.foo.text}")

class Baz(IBaz):
    def __init__(self, foo: IFoo):
        self.foo = foo

    def baz(self):
        print(f"I'm a Baz and my Foo says: {self.foo.text}")


In [3]:
# Create container and register services
container = ServiceContainer()

container.register(IFoo, Foo)
container.register(IBar, Bar)
container.register(IBaz, Baz)

In [4]:
# Retrieve services
bar = container.get(IBar)
baz = container.get(IBaz)

In [5]:
# Check default behaviour
bar.bar()
baz.baz()

I'm a Bar and my Foo says: default text
I'm a Baz and my Foo says: default text


In [6]:
# Now lets change Bar's Foo text
bar.foo.text = "new text"

bar.bar()
baz.baz()


I'm a Bar and my Foo says: new text
I'm a Baz and my Foo says: default text


We can see from the outputs, that after we modified the `Bar` service's `Foo.text`, the `Bar` output the new text, but the `Baz` still had the default text. This is because `Bar` and `Baz` each have their own instance of `Foo`, because it was registered with the `ServiceContainer` with a "transient" lifetime (the default).

We can specify a services lifetime when it is registered with the `ServiceContainer`, by passing it a `ServiceLife`.

There are a handful of lifetimes which we can specify:

 - `ServiceLife.TRANSIENT`
   This is the default lifetime. Each time the service is requested, a new instance is created.

 - `ServiceLife.SCOPED`
   A single service is created per "scope".

 - `ServiceLife.SINGLETON`
   A single service is created per container.

We have looked a little at "transient" behavior, so let's examine the others: "scoped" and "singleton".

We'll start with "singleton" as it's a little simpler than "scoped".

The "singleton" lifetime means that the `ServiceContainer` will only ever create one instance of the specified service. This means when multiple services define it as a dependency, they will all recieve the same object.  

If we update the previous example, and register `IFoo` as a "singleton" (rather than "transient"), our `IBar` and `IBaz` services should both get the same `IFoo`. This means, if one of the services modifies `IFoo`, it will be observed by the other.

In [7]:
from pyjudo import ServiceLife

container.unregister(IFoo)
container.register(IFoo, Foo, ServiceLife.SINGLETON)
# Or alternatively
# container.add_singleton(IFoo, Foo)

In [8]:
# Retrieve services
bar = container.get(IBar)
baz = container.get(IBaz)

In [9]:
# Check default behaviour
bar.bar()
baz.baz()

I'm a Bar and my Foo says: default text
I'm a Baz and my Foo says: default text


In [10]:
# Now lets change Bar's Foo text
bar.foo.text = "new text"

bar.bar()
baz.baz()

I'm a Bar and my Foo says: new text
I'm a Baz and my Foo says: new text


Now this time, both `bar` and `baz` output the new text.

> CAUTION: You should be careful when using singletons. Multiple objects reading from and/or writing to the same object can lead to unexpected outcomes, especially in multithreaded applications. It's beyond the scope of these documents to detail how to properly handle these cases, but you should be forewarned.

Lastly (kinda), there is the "scoped" lifetime. Services registered with a "scoped" lifetime have a single instance created per scope.

Service scopes are more complex and further detail on how they work are provided in [03 Scopes](03_scopes.ipynb), but a basic example is provided here.

You can create and use scopes with a context manager. So when you enter into the context a new scope is created, and it is disposed of when the context is exited.

```python
with container.create_scope() as scope:
    # This is a new scope
# aaand the scope is gone
```

In [11]:
from pyjudo import ServiceLife

container.unregister(IFoo)
container.register(IFoo, Foo, ServiceLife.SCOPED)
# Or alternatively
# `container.add_scoped(IFoo, Foo)`
# Going forward, we will use the more explicit registration methods:
# `add_transient`, `add_singleton`, and `add_scoped`

In [12]:
from pyjudo.exceptions import ServiceScopeError

# This will raise an error because we are not in a scope, and IFoo is scoped
try:
    bar = container.get(IBar)
except ServiceScopeError as e:
    print(f"Error: {e!r}")


with container.create_scope() as scope:
    print("\nIn a scope")
    bar = scope.get(IBar)
    baz = scope.get(IBaz)

    bar.foo.text = "im in a scope"

    bar.bar()
    baz.baz()

    with container.create_scope() as inner_scope:
        print("\nIn a nested scope")
        bar = inner_scope.get(IBar)
        baz = inner_scope.get(IBaz)

        bar.foo.text = "im in a nested scope"

        bar.bar()
        baz.baz()

    print("\nBack to the first scope")
    bar = scope.get(IBar)
    baz = scope.get(IBaz)

    bar.bar()
    baz.baz()

Error: ServiceScopeError('No scope available to resolve scoped services.')

In a scope
I'm a Bar and my Foo says: im in a scope
I'm a Baz and my Foo says: im in a scope

In a nested scope
I'm a Bar and my Foo says: im in a nested scope
I'm a Baz and my Foo says: im in a nested scope

Back to the first scope
I'm a Bar and my Foo says: im in a scope
I'm a Baz and my Foo says: im in a scope


In this example, we defined `IFoo` with a lifetime of "scoped".

First, we tried to retrieve an `IBar` service, outside of a scope. This will raise an exception, because there is no current scope.

> **NOTE**  
> It is debatable whether there should be a "global" scope, but I'm of the opinion that this is a poor design.  
>  
> Services which are scoped imply they have an expiration and should be disposed of. A "global" scope would not be disposed of until the container is disposed, meaning scoped services resolved in the "global" scope are effectively implicit singletons.  
>  
> I think it's better to explicitly define that behaviour to prevent accidental misuse; thus, scoped services cannot be resolved unless within an explicitly defined scope.

We then created a new scope, and retrieved the `IBar` and `IBaz` services, and updated the `IFoo` text to "im in a scope".

A second, nested scope was then created and we retrieved the `IBar` and `IBaz` services from the nested scope and updated the `IFoo` text again, this time to "im in a nested scope"

Finally, we left the nested scope, and retrieved the services again (somewhat unnecessarily, but helps prove the theory...), and checked the text.

You should notice that when we finally checked the `IFoo` text after leaving the nested scope, the text was the same as when we set first set it: "im in a scope". This is because each scope has it's own `IFoo` instance. 

I hope that gives a reasonable summary of the different service lifetimes and an introduction to their use. 

Scopes and scoped services are a little more complex, and are gone into further detail in the next notebook, [03 Scopes](03_scopes.ipynb).